# Azure AI Search กับการผสาน NVIDIA NIM และ LlamaIndex

ในโน้ตบุ๊กนี้ เราจะสาธิตวิธีการใช้โมเดล AI ของ NVIDIA และ LlamaIndex เพื่อสร้างกระบวนการ Retrieval-Augmented Generation (RAG) ที่ทรงพลัง เราจะใช้ LLMs และ embeddings ของ NVIDIA ผสานเข้ากับ Azure AI Search ในฐานะ vector store และดำเนินการ RAG เพื่อเพิ่มคุณภาพและประสิทธิภาพของการค้นหา

## ประโยชน์
- **ความสามารถในการขยาย**: ใช้โมเดลภาษาขนาดใหญ่ของ NVIDIA และ Azure AI Search เพื่อการค้นหาที่มีประสิทธิภาพและสามารถขยายได้
- **ความคุ้มค่า**: ปรับปรุงการค้นหาและการดึงข้อมูลด้วย vector storage ที่มีประสิทธิภาพและเทคนิคการค้นหาแบบไฮบริด
- **ประสิทธิภาพสูง**: ผสาน LLMs ที่ทรงพลังเข้ากับการค้นหาแบบเวกเตอร์เพื่อการตอบสนองที่รวดเร็วและแม่นยำยิ่งขึ้น
- **คุณภาพ**: รักษาคุณภาพการค้นหาให้สูงโดยการเชื่อมโยงคำตอบของ LLM กับเอกสารที่เกี่ยวข้องที่ถูกดึงมา

## สิ่งที่ต้องเตรียม
- 🐍 Python 3.9 หรือสูงกว่า
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API Key สำหรับการเข้าถึง LLMs และ Embeddings ของ NVIDIA ผ่านบริการไมโครเซอร์วิส NVIDIA NIM

## ฟีเจอร์ที่ครอบคลุม
- ✅ การผสาน NVIDIA LLM (เราจะใช้ [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA Embeddings (เราจะใช้ [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ โหมดการค้นหาขั้นสูงของ Azure AI Search
- ✅ การสร้างดัชนีเอกสารด้วย LlamaIndex
- ✅ RAG โดยใช้ Azure AI Search และ LlamaIndex ร่วมกับ NVIDIA LLMs

มาเริ่มกันเลย!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## การติดตั้งและข้อกำหนด  
สร้างสภาพแวดล้อม Python โดยใช้ Python เวอร์ชัน >3.10  

## เริ่มต้นใช้งาน!  


ในการเริ่มต้น คุณจำเป็นต้องมี `NVIDIA_API_KEY` เพื่อใช้งานโมเดล NVIDIA AI Foundation:
1) สร้างบัญชีฟรีกับ [NVIDIA](https://build.nvidia.com/explore/discover).
2) คลิกเลือกโมเดลที่คุณต้องการ.
3) ใต้หัวข้อ Input ให้เลือกแท็บ Python และคลิก **Get API Key** จากนั้นคลิก **Generate Key**.
4) คัดลอกและบันทึกคีย์ที่สร้างขึ้นในชื่อ NVIDIA_API_KEY. จากนั้นคุณจะสามารถเข้าถึง endpoints ได้.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## ตัวอย่าง RAG โดยใช้ LLM และ Embedding
### 1) เริ่มต้น LLM
`llama-index-llms-nvidia` หรือที่รู้จักกันในชื่อ NVIDIA's LLM connector ช่วยให้คุณสามารถเชื่อมต่อและสร้างผลลัพธ์จากโมเดลที่เข้ากันได้ซึ่งมีอยู่ใน NVIDIA API catalog ดูรายการโมเดลสำหรับการตอบกลับแบบแชทได้ที่นี่: https://build.nvidia.com/search?term=Text-to-Text

ในที่นี้เราจะใช้ **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) เริ่มต้นการฝังข้อมูล  
`llama-index-embeddings-nvidia` หรือที่รู้จักกันในชื่อ NVIDIA's Embeddings connector ช่วยให้คุณสามารถเชื่อมต่อและสร้างจากโมเดลที่เข้ากันได้ซึ่งมีอยู่ใน NVIDIA API catalog เราเลือกใช้ `nvidia/nv-embedqa-e5-v5` เป็นโมเดลการฝังข้อมูล ดูรายการโมเดลการฝังข้อความได้ที่นี่: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) สร้าง Query Engine เพื่อถามคำถามเกี่ยวกับข้อมูลของคุณ

นี่คือตัวอย่างการค้นหาโดยใช้การค้นหาแบบเวกเตอร์ใน Azure AI Search และการเชื่อมโยงคำตอบกับ LLM ของเรา (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

นี่คือการค้นหาโดยใช้การค้นหาแบบไฮบริดใน Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### การวิเคราะห์การค้นหาแบบเวกเตอร์
การตอบกลับของ LLM สามารถจับประเด็นสำคัญเกี่ยวกับผลกระทบทางเศรษฐกิจที่ระบุไว้ในข้อความต้นฉบับเกี่ยวกับตลาดหุ้นของรัสเซียได้อย่างถูกต้อง โดยเฉพาะอย่างยิ่งได้กล่าวว่าตลาดหุ้นรัสเซียประสบกับการลดลงอย่างมาก สูญเสียมูลค่าไปถึง 40% และการซื้อขายถูกระงับเนื่องจากสถานการณ์ที่เกิดขึ้น การตอบกลับนี้สอดคล้องกับข้อมูลที่ให้ไว้ในข้อความต้นฉบับ ซึ่งแสดงให้เห็นว่า LLM สามารถระบุและสรุปรายละเอียดที่เกี่ยวข้องเกี่ยวกับผลกระทบต่อตลาดหุ้นจากการกระทำของรัสเซียและมาตรการคว่ำบาตรที่ถูกนำมาใช้ได้อย่างถูกต้อง

#### ความคิดเห็นเกี่ยวกับโหนดต้นทาง
โหนดต้นทางให้ข้อมูลเชิงลึกเกี่ยวกับผลกระทบทางเศรษฐกิจที่รัสเซียเผชิญเนื่องจากมาตรการคว่ำบาตรระหว่างประเทศ ข้อความระบุว่าตลาดหุ้นรัสเซียสูญเสียมูลค่าไปถึง 40% และการซื้อขายถูกระงับ นอกจากนี้ยังกล่าวถึงผลกระทบทางเศรษฐกิจอื่น ๆ เช่น การลดค่าเงินรูเบิล และการแยกตัวทางเศรษฐกิจของรัสเซียในวงกว้าง การตอบกลับของ LLM สามารถสรุปประเด็นสำคัญจากโหนดเหล่านี้ได้อย่างมีประสิทธิภาพ โดยมุ่งเน้นไปที่ผลกระทบต่อตลาดหุ้นตามที่คำถามร้องขอ


ตอนนี้ มาดูตัวอย่างคำถามที่การค้นหาแบบ Hybrid Search ไม่สามารถให้คำตอบที่มีเหตุผลได้:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### การค้นหาแบบไฮบริด: การวิเคราะห์การตอบกลับของ LLM
การตอบกลับของ LLM ในตัวอย่างการค้นหาแบบไฮบริดแสดงให้เห็นว่าบริบทที่ให้มานั้นไม่ได้ระบุวันที่ที่แน่นอนของการรุกรานยูเครนโดยรัสเซีย การตอบกลับนี้บ่งบอกว่า LLM กำลังใช้ข้อมูลที่มีอยู่ในเอกสารต้นทาง แต่ยอมรับว่าขาดรายละเอียดที่ชัดเจนในข้อความ

การตอบกลับนี้ถูกต้องในการระบุว่าบริบทกล่าวถึงเหตุการณ์ที่เกี่ยวข้องกับการรุกรานของรัสเซีย แต่ไม่ได้ระบุวันที่ของการรุกรานอย่างชัดเจน สิ่งนี้แสดงให้เห็นถึงความสามารถของ LLM ในการเข้าใจข้อมูลที่ให้มา พร้อมทั้งรับรู้ถึงช่องว่างในเนื้อหา LLM กระตุ้นให้ผู้ใช้ค้นหาแหล่งข้อมูลภายนอกหรือบันทึกทางประวัติศาสตร์เพื่อหาวันที่ที่แน่นอน ซึ่งแสดงถึงความรอบคอบเมื่อข้อมูลไม่สมบูรณ์

### การวิเคราะห์โหนดต้นทาง
โหนดต้นทางในตัวอย่างการค้นหาแบบไฮบริดมีข้อความที่ตัดตอนมาจากสุนทรพจน์ที่กล่าวถึงการตอบสนองของสหรัฐฯ ต่อการกระทำของรัสเซียในยูเครน โหนดเหล่านี้เน้นถึงผลกระทบทางภูมิรัฐศาสตร์ในวงกว้างและขั้นตอนที่สหรัฐฯ และพันธมิตรดำเนินการเพื่อตอบสนองต่อการรุกราน แต่ไม่ได้กล่าวถึงวันที่ของการรุกรานโดยเฉพาะ สิ่งนี้สอดคล้องกับการตอบกลับของ LLM ซึ่งระบุอย่างถูกต้องว่าบริบทขาดข้อมูลวันที่ที่ชัดเจน


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### การวิเคราะห์ Hybrid w/Reranking: การตอบสนองของ LLM
ในตัวอย่าง Hybrid w/Reranking การตอบสนองของ LLM ให้บริบทเพิ่มเติมโดยระบุว่าเหตุการณ์เกิดขึ้นหกวันก่อนการกล่าวสุนทรพจน์ ซึ่งแสดงให้เห็นว่า LLM สามารถอนุมานวันที่การรุกรานได้จากช่วงเวลาของการกล่าวสุนทรพจน์ แม้ว่าจะยังต้องทราบวันที่ที่แน่นอนของการกล่าวสุนทรพจน์เพื่อความแม่นยำ

การตอบสนองนี้แสดงให้เห็นถึงความสามารถที่ดีขึ้นในการใช้เบาะแสจากบริบทเพื่อให้คำตอบที่ให้ข้อมูลมากขึ้น โดยเน้นข้อได้เปรียบของการจัดลำดับใหม่ (reranking) ซึ่งช่วยให้ LLM เข้าถึงและจัดลำดับความสำคัญของข้อมูลที่เกี่ยวข้องมากขึ้น เพื่อให้ได้รายละเอียดที่ใกล้เคียงกับสิ่งที่ต้องการ (เช่น วันที่การรุกราน)

### การวิเคราะห์ Source Nodes
Source nodes ในตัวอย่างนี้รวมถึงการอ้างอิงถึงช่วงเวลาของการรุกรานของรัสเซีย โดยเฉพาะการกล่าวถึงว่าเหตุการณ์เกิดขึ้นหกวันก่อนการกล่าวสุนทรพจน์ แม้ว่าจะยังไม่ได้ระบุวันที่ที่แน่นอน แต่ nodes เหล่านี้ให้บริบทเชิงเวลา ซึ่งช่วยให้ LLM สามารถตอบสนองได้อย่างละเอียดและมีความหมายมากขึ้น การเพิ่มรายละเอียดนี้แสดงให้เห็นว่า reranking สามารถปรับปรุงความสามารถของ LLM ในการดึงและอนุมานข้อมูลจากบริบทที่ให้มา ส่งผลให้ได้คำตอบที่แม่นยำและให้ข้อมูลมากขึ้น


**หมายเหตุ:**  
ในโน้ตบุ๊กนี้ เราใช้ไมโครเซอร์วิส NVIDIA NIM จาก NVIDIA API Catalog  
API ที่กล่าวถึงข้างต้น ได้แก่ `NVIDIA (llms)`, `NVIDIAEmbedding` และ [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview)  
โปรดทราบว่า API เหล่านี้ยังสามารถรองรับไมโครเซอร์วิสที่โฮสต์ด้วยตัวเองได้เช่นกัน  

**ตัวอย่าง:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
